# Домашнее задание №4

## Задача №0 (3)
Дорешайте задачу про риды и BWT из файла practice4.ipynb. В ответе помимо кода укажите количество ридов первой, второй и третьей категории, а также вкратце опишите ход решения.

In [ ]:
!pip install Bio

In [ ]:
!pip install pysuffixarray

In [ ]:
from pysuffixarray.core import SuffixArray

class BWTSearcher:
    def __init__(self, reference):
        # Construct the suffix array
        self.sa = SuffixArray(reference)

        # Construct the BWT from the suffix array
        self.bwt_text = ''.join(reference[i-1] if i != 0 else '$' for i in self.sa.suffix_array())

        self.Occ = {ch: [0] * len(self.bwt_text) for ch in "$ACGT"}

        for i, ch in enumerate(self.bwt_text):
            for ch2 in "ACGT":
                if i != 0:
                    self.Occ[ch2][i] = self.Occ[ch2][i - 1]
            if ch in self.Occ.keys():
                self.Occ[ch][i] += 1

        self.count = {}

        total = 0
        for ch in "$ACGT":
            self.count[ch] = total
            total += self.bwt_text.count(ch)


    def bwt_pattern_search(self, pattern, mismatches, *kwargs):
        if kwargs == ():
          top = 0
          bottom = len(self.bwt_text) - 1
        else:
          top, bottom = kwargs
        if mismatches == 0:
          for char in reversed(pattern):
              if char in self.Occ:
                  top = self.count[char] + (0 if top == 0 else self.Occ[char][top  - 1])
                  bottom = self.count[char] + self.Occ[char][bottom] - 1
              else:
                  return []
              if top > bottom:
                  return []
          return self.sa.suffix_array()[top:bottom + 1]
        else:
            for char in 'AGTC':
              new_top = self.count[char] + (0 if top == 0 else self.Occ[char][top  - 1])
              new_bottom = self.count[char] + self.Occ[char][bottom] - 1
              if new_top <= new_bottom:
                  if len(pattern) == 0:
                      return self.sa.suffix_array()[new_top:new_bottom + 1]
                  else:
                      mm = mismatches
                      if char != pattern[-1]:
                            mm = max(0, mismatches - 1)
                      tri = self.bwt_pattern_search(pattern[:-1], mm, new_top, new_bottom)
                      if not tri is None and tri != []:
                          return tri



In [ ]:
from Bio import SeqIO

m_0 = 0
m_1 = 0
m_5 = 0

with open("BWT_folder/genome.fa", "r") as file:
    bwts = BWTSearcher(next(SeqIO.parse(file, "fasta")).seq)

# Process each read through the BWTSearcher
with open("BWT_folder/sample_reads.fasta", "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        read_sequence = str(record.seq)
        mm = 0
        # Here is just a placeholder to demonstrate using the read with the BWTSearcher
        res = bwts.bwt_pattern_search(read_sequence, mm)
        if res == []:
            mm = 1
            res = bwts.bwt_pattern_search(read_sequence, mm)
            if res is None:
                mm = 5
                res = bwts.bwt_pattern_search(read_sequence, mm)
                if res == []:
                    print('error')
                m_5 += 1
            else:
                m_1 += 1
        else:
            m_0 += 1
    print(f'No mismathes: {m_0}; one mismatch: {m_1}; five mismatches: {m_5}')

No mismathes: 45; one mismatch: 31; five mismatches: 24


Если ошибки не предусмотрены - классический поиск по BWT. Если предусмотрены n ошибок, будем пробовать подставлять все нуклеотиды по очереди в строку-паттерн с конца. Если нуклеотид совпадает с действительным на данном месте, то уменьшать количество возможных ошибок не будем (поскольку на данном шаге ошибочных нуклеотидов не прибавилось), иначе уменьшим количество на единицу. В обоих случаях далее рассматриваем строку до этого нуклеотида (аналогично классическому алгоритму с помощью top и bottom учитывая, что начиная с этого нуклеотида строку с m <= n ошибками нашли). Таким образом, проверяем все возможные подпоследовательности, для позиции паттерна (с конца, пока возможно) рекурсивно вызываем алгоритм для 4 или менее возможных вариантов, в зависимости от того, есть ли подпоследовательность, начинающаяся с данного нуклеотида (условие top <= bottom). Как только в собирающейся подпоследовательности набралось n ошибок - переходим к поиску для 0.


Это домашнее задание можно выполнять целиком в этом ноутбуке, либо алгоритмы написать в отдельном файле и импортировать сюда, для использования. В папке data лежат два файла islands.fasta и nonIslands.fasta. В них хранятся прочтения из CpG островков и из обычных участков генома соответственно, этими данными нужно будет воспользоваться в первом задании.

## Задача №1 (1)
Определите частоты генерации для каждого из нуклеотидов внутри CpG островков и вне их. Посчитайте так-же частоты для всех упорядоченных пар нуклеотидов и сравните частоту пары CG внутри островков и снаружи. Сделайте вывод.

Частоты генерации каждого нуклеотида внутри CpG и вне:

In [ ]:
with open("data/islands.fasta", "r") as file:
    islands = ' '.join([str(record.seq) for record in SeqIO.parse(file, "fasta")])
with open("data/nonIslands.fasta", "r") as file:
    non_islands = ' '.join([str(record.seq) for record in SeqIO.parse(file, "fasta")])
n_i = len(islands) - islands.count(' ')
n_ni = len(non_islands)

In [ ]:
probs_i = []
probs_ni = []
for ch in 'AGTC':
    p_i = islands.count(ch) / n_i
    probs_i.append(p_i)
    print(f'P({ch}|CpG) = ', p_i)
    p_ni = non_islands.count(ch) / n_ni
    probs_ni.append(p_ni)
    print(f'P({ch}|non CpG) = ', p_ni)

P(A|CpG) =  0.20246225382932168
P(A|non CpG) =  0.28929861153616176
P(G|CpG) =  0.29827680525164113
P(G|non CpG) =  0.1984481597792771
P(T|CpG) =  0.20286433260393874
P(T|non CpG) =  0.3161168211758391
P(C|CpG) =  0.29639660831509845
P(C|non CpG) =  0.19613640750872205


In [ ]:
pairs = {ch_1 + ch_2 for ch_1 in 'CGTA' for ch_2 in 'GCTA'}

Частоты генерации пар нуклеотидов внутри и вне CpG:

In [ ]:
#количество способов взять пару идущих подряд нуклеотидов из последовательности - n - 1
for ch in pairs:
    print(f'P({ch}|CpG) = ', islands.count(ch) / (n_i - 1))
    print(f'P({ch}|non CpG) = ', non_islands.count(ch) / (n_ni - 1))

P(GT|CpG) =  0.04663540662894054
P(GT|non CpG) =  0.05387208557761121
P(CT|CpG) =  0.07171309401342835
P(CT|non CpG) =  0.07265810388338544
P(CA|CpG) =  0.06286188809132606
P(CA|non CpG) =  0.06818524238201062
P(GA|CpG) =  0.05984465531855999
P(GA|non CpG) =  0.06023296546009266
P(TA|CpG) =  0.02843955920119234
P(TA|non CpG) =  0.07100549926550906
P(TT|CpG) =  0.038568938339425145
P(TT|non CpG) =  0.07774774944442352
P(AG|CpG) =  0.07188842228895577
P(AG|non CpG) =  0.07019096764473239
P(TC|CpG) =  0.05949974275157077
P(TC|non CpG) =  0.06475761798937812
P(AT|CpG) =  0.03340099381865257
P(AT|non CpG) =  0.08209819578891861
P(CG|CpG) =  0.05895433778838561
P(CG|non CpG) =  0.0052779765716222836
P(AA|CpG) =  0.03869886178852894
P(AA|non CpG) =  0.06641022260725452
P(GG|CpG) =  0.0767155570885003
P(GG|non CpG) =  0.04026987833816716
P(GC|CpG) =  0.08815620573200375
P(GC|non CpG) =  0.03422916117367886
P(CC|CpG) =  0.07628913465238912
P(CC|non CpG) =  0.04031225281554861
P(AC|CpG) =  0.045

Частоту пары CG внутри островков примерно в 11 раз больше чем снаружи. Концентрация таких пар сравнительно выше для островков, чем для других участков генома.


## Задача №2 (2)
Напишите марковскую модель, которая имеет открытые состояния {A, T, G, C}, и скрытые состояния {+, -}. Когда модель в состоянии **+**, то вероятность генерации некоторого символа нуклеотида соответствует его частоте внутри CpG островков, вычислиному в первом задании, если состояние **-**, то частоте вне островков. Вероятность остаться внутри островка 0.95, а перейти в обычный геном 0.05. Для остальной части генома соответствующие вероятности 0.995 и 0.005. Саму модель можно реализовать в виде итератора, определив метод next, который возвращает пару - состояние и нуклеотид, который в этом состоянии произведен.    
Воспользуйтесь данной моделью для того чтобы сгенерировать набор из 20 последовательностей длинной от 1 000 до 100 000, причем к каждой последовательности должна прилагаться последовательность состояний.

In [ ]:
import random
nucleotides = 'A G T C'.split()

def hmm(seq_len):

    is_cpg = random.choice([True, False])

    for _ in range(seq_len):
        probs = probs_i if is_cpg else probs_ni
        state = '+' if is_cpg else '-'
        nucleotide = random.choices(nucleotides, probs, k=1)[0]
        yield (state, nucleotide)
        should_switch = random.choices([False, True], [0.95, 0.05] if is_cpg else [0.995, 0.005], k=1)[0]
        if should_switch:
            is_cpg = not is_cpg

In [ ]:
hmm_res = [[x for x in hmm(random.randint(1000, 100000))] for _ in range(20)]

In [ ]:
seqs = [[nucleotide for state, nucleotide in chain] for chain in hmm_res]
states = [[state for state, nucleotide in chain] for chain in hmm_res]
print(seqs[0])
print(states[0])

['A', 'A', 'T', 'A', 'G', 'T', 'T', 'A', 'T', 'T', 'G', 'G', 'T', 'T', 'A', 'G', 'G', 'G', 'T', 'T', 'T', 'A', 'A', 'A', 'G', 'A', 'T', 'T', 'T', 'G', 'A', 'T', 'C', 'G', 'A', 'G', 'A', 'G', 'G', 'C', 'T', 'A', 'A', 'A', 'T', 'G', 'A', 'T', 'T', 'G', 'C', 'T', 'T', 'T', 'A', 'G', 'C', 'A', 'T', 'G', 'A', 'T', 'G', 'A', 'G', 'T', 'G', 'T', 'T', 'T', 'G', 'A', 'A', 'G', 'A', 'A', 'A', 'T', 'T', 'G', 'T', 'C', 'A', 'A', 'A', 'A', 'C', 'C', 'G', 'C', 'T', 'A', 'T', 'G', 'A', 'T', 'C', 'G', 'T', 'G', 'C', 'A', 'A', 'C', 'G', 'A', 'G', 'T', 'A', 'T', 'A', 'A', 'G', 'A', 'A', 'G', 'T', 'T', 'C', 'G', 'G', 'A', 'T', 'T', 'A', 'T', 'G', 'A', 'T', 'T', 'A', 'T', 'C', 'G', 'A', 'G', 'G', 'G', 'C', 'A', 'A', 'C', 'C', 'T', 'C', 'G', 'A', 'T', 'G', 'C', 'G', 'G', 'C', 'G', 'T', 'G', 'A', 'A', 'C', 'C', 'G', 'T', 'A', 'G', 'C', 'T', 'C', 'G', 'C', 'C', 'G', 'G', 'G', 'C', 'T', 'C', 'T', 'A', 'C', 'A', 'G', 'C', 'T', 'C', 'T', 'A', 'T', 'G', 'T', 'G', 'A', 'G', 'T', 'T', 'T', 'G', 'C', 'T', 'C', 'T',

## Задача №3 (4)
Напишите алгоритм Витерби для восстановления последовательности скрытых состояний марковской модели из второго задаания. Воспользуйтесь им, воссстановив состояния тех последовательностей, которые вы получили во втором задании и посчитайте TP, TN, FP, FN по количеству правильно или ошибочно предсказанных позиций из CpG остравков.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from math import log

def viterbi(seq_sym, T, E, S, P): #transition, emitions, possible states, start probs

    seq_to_num = {x: i for i, x in enumerate('A G T C'.split())}
    seq = [seq_to_num[x] for x in seq_sym]
    cur = seq[0]
    n = len(seq)
    m = len(S)
    t_state = [[log(P[x]) + log(E[x][cur]) for x in range(m)]]
    t_ind = [[0] * m]

    for t in range(1, n):
        t_state.append([0] * m)
        t_ind.append([0] * m)
        cur = seq[t]

        for k in range(m):
            t_ind[t][k] = np.argmax([t_state[t - 1][i] + log(T[i][k]) + log(E[k][cur]) for i in range(m)])
            t_state[t][k] = t_state[t - 1][t_ind[t][k]] + log(T[t_ind[t][k]][k]) + log(E[k][cur])

    pred = [np.argmax(t_state[n - 1])]
    for i in range(n - 1, 0, -1):
        pred.append(t_ind[i][pred[n - 1 - i]])
    pred = pred[::-1]
    pred = [S[x] for x in pred]

    return pred

T = [[0.95, 0.05], [0.005, 0.995]]  # + -
E = [probs_i, probs_ni]
S = ['+', '-']
P = [0.5, 0.5]

for seq, chain in zip(seqs, states):
    pred_chain = viterbi(seq, T, E, S, P)
    print(f'Seq of lenght {len(seq)}')
    print(confusion_matrix(chain, pred_chain))
    print()

Seq of lenght 34506
[[  187  2965]
 [   19 31335]]

Seq of lenght 24941
[[  123  2268]
 [    0 22550]]

Seq of lenght 75715
[[  414  7173]
 [   75 68053]]

Seq of lenght 40310
[[  188  3872]
 [   19 36231]]

Seq of lenght 40860
[[  163  3238]
 [   12 37447]]

Seq of lenght 9388
[[ 150  914]
 [   6 8318]]

Seq of lenght 62530
[[    0  5459]
 [    0 57071]]

Seq of lenght 51832
[[  277  4707]
 [   65 46783]]

Seq of lenght 60514
[[  207  5324]
 [   39 54944]]

Seq of lenght 15022
[[   72  1333]
 [    1 13616]]

Seq of lenght 11828
[[  137   770]
 [   12 10909]]

Seq of lenght 1999
[[   0  267]
 [   0 1732]]

Seq of lenght 73706
[[  249  6781]
 [   53 66623]]

Seq of lenght 40422
[[  390  3778]
 [   78 36176]]

Seq of lenght 97166
[[  453  7999]
 [  116 88598]]

Seq of lenght 44573
[[  177  3996]
 [    1 40399]]

Seq of lenght 14648
[[   41  1387]
 [    5 13215]]

Seq of lenght 11504
[[   93   920]
 [   70 10421]]

Seq of lenght 48578
[[  367  4427]
 [   55 43729]]

Seq of lenght 45116
[[

## Задача №4 (4)
Напишите алгоритм вперед назад для модели из второго задания. Пользуясь этим алгоритмом найдите вероятности того, что модель находилась в состоянии **+** для каждой позиции строк из второго задания. Устанавливая различные пороговые значения, определите позиции соответствующие CpG островкам и посчитайте TP. Постройте график зависимости TP от выбранного порогового значения. Есть ли пороговые значения при которых TP больше чем в задании №3?